In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
import torchmetrics
#import scgen
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.spatial.distance import cdist
import warnings
import helper_scgen as hscg
import sys
print(sys.executable)
import scgen

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score
import anndata as ad

In [ ]:
pre = "MH59"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

### MH50_c
proof of principle - split according to replicates

In [ ]:
cyno_col = '#178f92'
cyno_shifted_col = '#D8C29D'
human_col = '#094568'

palette_species_shifted = [cyno_col,cyno_shifted_col,human_col]

In [ ]:
model_path = os.path.join(*["F:\\","monkey_IZI","analysis","cluster_runs","workspace-runVAE",
                            "output","MH50_c","models"])
model_dir_names = os.listdir(model_path)

warnings.filterwarnings("ignore")
model_dict = {}
for model_dir_name in model_dir_names:
    load_string = os.path.join(model_path,model_dir_name,'MH50_c_scgen_model_' + str(model_dir_name)+'.pt')
    model = scgen.SCGEN.load(load_string)
    #df_ct = model.history['train_loss_epoch'].join(model.history['validation_loss'])
    model_dict[model_dir_name] = model

In [ ]:
'''
for celltype in [model_dir_names[0]]:
    model = model_dict[celltype]
    adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    adata_model = model.adata
    latent_adata_train = hscg.get_latent_representation_object(model=model,adata= adata_model)
    
    #train set
    latent_adata_train_human_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train,
                                                                   col_names=['timepoint','species'],
                                                                   val_names=['00hr','human'])
    latent_adata_train_cyno_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train,
                                                                   col_names=['timepoint','species'],
                                                                   val_names=['00hr','cyno'])
    #get species shift
    delta_cross_species_train = hscg.get_delta_in_latent_space(adata_lat_base=latent_adata_train_cyno_00hr,
                                  adata_lat_dest=latent_adata_train_human_00hr)
'''

In [ ]:
base_anndata_objects_MH50_c = os.path.join('F:\\monkey_IZI\\analysis','cluster_runs',
                                    'workspace-runVAE','output','MH50_c','anndata_objects')

In [ ]:
adata_train_all_ct = sc.read_h5ad(os.path.join(base_anndata_objects_MH50_c,'MH50_c_adata_train.h5ad'))
adata_test_all_ct = sc.read_h5ad(os.path.join(base_anndata_objects_MH50_c,'MH50_c_adata_test.h5ad'))

adata_train_all_ct.obs.rename(columns={'species_x': 'species'}, inplace=True)
adata_test_all_ct.obs.rename(columns={'species_x': 'species'}, inplace=True)

adata_train_all_ct.obs['cluster_azimut1_5_scanvi_v2'] = adata_train_all_ct.obs['cluster_azimut1_5_scanvi'].replace(
    {'NK': 'NK+Proliferating', 'NK Proliferating': 'NK+Proliferating'})

adata_test_all_ct.obs['cluster_azimut1_5_scanvi_v2'] = adata_test_all_ct.obs['cluster_azimut1_5_scanvi'].replace(
    {'NK': 'NK+Proliferating', 'NK Proliferating': 'NK+Proliferating'})

celltypes = ['B',
 'CD14 Mono',
 'CD16 Mono',
 'CD4 T',
 'CD8 T',
 'MAIT',
 'NK+Proliferating']

In [ ]:
for celltype in celltypes:
    model = model_dict[celltype]
    adata = model.adata
    adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    latent_adata_train = hscg.get_latent_representation_object(model=model,adata= adata)
    
    #train set
    latent_adata_train_human_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train,
                                                                   col_names=['timepoint','species'],
                                                                   val_names=['00hr','human'])
    latent_adata_train_cyno_00hr = hscg.filter_multiple_adata_obs(adata=latent_adata_train,
                                                                   col_names=['timepoint','species'],
                                                                   val_names=['00hr','cyno'])
    #get species shift
    delta_cross_species_train = hscg.get_delta_in_latent_space(adata_lat_base=latent_adata_train_cyno_00hr,
                                  adata_lat_dest=latent_adata_train_human_00hr)
    set_type = 'train'
    adata_train_00hr = hscg.filter_multiple_adata_obs(adata=adata_train_all_ct,
                                                         col_names=['cluster_azimut1_5_scanvi_v2','timepoint'],
                                                         val_names=[celltype,'00hr'])
    adata_train_00hr_cyno = hscg.filter_multiple_adata_obs(adata=adata_train_all_ct,col_names=
                                                           ['cluster_azimut1_5_scanvi_v2','timepoint','species'],
                                                          val_names=
                                                           [celltype,'00hr','cyno'])
    adata_train_00hr_human = hscg.filter_multiple_adata_obs(adata=adata_train_all_ct,col_names=
                                                            ['cluster_azimut1_5_scanvi_v2','timepoint','species'],
                                                          val_names=[celltype,'00hr','human'])

    latent_adata_train_00hr = hscg.get_latent_representation_object(model=model,adata=adata_train_00hr)
    latent_adata_train_00hr_human = hscg.filter_multiple_adata_obs(adata=latent_adata_train_00hr ,
                                                                       col_names=['timepoint','species'],
                                                                       val_names=['00hr','human'])
    latent_adata_train_00hr_cyno = hscg.filter_multiple_adata_obs(adata=latent_adata_train_00hr ,
                                                                   col_names=['timepoint','species'],
                                                                   val_names=['00hr','cyno'])

    latent_adata_train_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_train_00hr_cyno,
                                                                           delta=delta_cross_species_train,
                                                                           cols_suffix=['timepoint','species'],
                                                                           suffix=['_shifted','_shifted'])

    adata_latent_comp_train = ad.concat([latent_adata_train_00hr_human,latent_adata_train_00hr_cyno,latent_adata_train_00hr_cyno_shifted])
    h.prepare_umap(adata_latent_comp_train)
    h.umap_plot_v2(adata=adata_latent_comp_train,
                 color='species',
                 base_plots_path = base_plots_path,
                 plot_name = pre +'_'+ set_type +'_umap_latent' + str(celltype) + '.pdf',
                palette = palette_species_shifted) 

    latent_adata_train_00hr_cyno = hscg.get_latent_representation_object(model=model,adata= adata_train_00hr_cyno)

    latent_adata_train_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_train_00hr_cyno,delta=delta_cross_species_train)

    decoded_adata_train_00hr_cyno_shifted =hscg.decode(model=model,
                                              latent_adata=latent_adata_train_00hr_cyno_shifted,
                                              input_adata=adata_train_00hr_cyno) 
    decoded_adata_train_00hr_cyno_shifted.obs['timepoint'] = decoded_adata_train_00hr_cyno_shifted.obs['timepoint'].astype(str) + '_shifted'
    decoded_adata_train_00hr_cyno_shifted.obs['species'] = decoded_adata_train_00hr_cyno_shifted.obs['species'].astype(str) + '_shifted'
    adata_train_00hr_decoded = ad.concat([adata_train_00hr_cyno,adata_train_00hr_human,decoded_adata_train_00hr_cyno_shifted])
    h.prepare_umap(adata_train_00hr_decoded)


    h.umap_plot_v2(adata=adata_train_00hr_decoded,
                 color='species',
                 base_plots_path = base_plots_path,
                 plot_name = pre +'_'+ set_type +'_umap_GE_' + str(celltype) + '.pdf',
                palette = palette_species_shifted) 

    mean_expression_00hr_cyno_train = np.array(adata_train_00hr_cyno.X.mean(axis=0)).flatten()
    mean_expression_00hr_human_train = np.array(adata_train_00hr_human.X.mean(axis=0)).flatten()
    mean_expression_00hr_cyno_shifted_train = np.array(decoded_adata_train_00hr_cyno_shifted.X.mean(axis=0)).flatten()
    index_name_train = ['mean_expression_00hr_cyno_train','mean_expression_00hr_human_train','mean_expression_00hr_cyno_shifted_train']
    df_ct_mean_expr = pd.DataFrame([ mean_expression_00hr_cyno_train,
                                    mean_expression_00hr_human_train,
                                    mean_expression_00hr_cyno_shifted_train],index = index_name_train)

    df_ct_mean_expr.to_csv(os.path.join(base_table_path,pre + '_df_ct_mean_expr_train_' + str(celltype) + '.csv'))
    plt.figure(figsize=(6, 4.5))
    plt.scatter(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_human_train,c='#9eb4e0',edgecolors='k',alpha=0.7,s=50)
    plt.xlabel("Cyno Humanized", fontsize=12)
    plt.ylabel("Human", fontsize=12)
    plt.title('Mean correlation plot - ' + str(celltype), fontsize=14)
    plt.savefig(os.path.join(base_plots_path,pre +'_' +set_type +'_scatter_correlation_human_cyno_' + str(celltype) + '.pdf'),bbox_inches='tight')
    #plt.show()
    plt.close()

    plt.figure(figsize=(6, 4.5))
    plt.scatter(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_cyno_train,c='#9eb4e0',edgecolors='k',alpha=0.7,s=50)
    plt.xlabel("Cyno Humanized", fontsize=12)
    plt.ylabel("Cyno", fontsize=12)
    plt.title('Mean correlation plot - ' + str(celltype), fontsize=14)
    plt.savefig(os.path.join(base_plots_path,pre +'_' +set_type +'_scatter_correlation_cyno_cyno_' + str(celltype) + '.pdf'),bbox_inches='tight')
    #plt.show()
    plt.close()

    #R2 score
    r2_cyno_shifted_human_train = r2_score(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_human_train)
    r2_cyno_shifted_cyno_train = r2_score(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_cyno_train)
    r2_cyno_human_train = r2_score(mean_expression_00hr_cyno_train,mean_expression_00hr_human_train)

    columns_df = ['r2_cyno_shifted_human','r2_cyno_shifted_cyno','r2_cyno_human','celltype','set_type']
    df_r2_humanization_train = pd.DataFrame(columns = columns_df)
    df_r2_humanization_train  = df_r2_humanization_train.append(pd.DataFrame([r2_cyno_shifted_human_train,r2_cyno_shifted_cyno_train,r2_cyno_human_train,celltype,set_type],index=columns_df).T)

    mean_expression_00hr_cyno_train = np.array(adata_train_00hr_cyno.X.mean(axis=0)).flatten()
    mean_expression_00hr_human_train = np.array(adata_train_00hr_human.X.mean(axis=0)).flatten()
    mean_expression_00hr_cyno_shifted_train = np.array(decoded_adata_train_00hr_cyno_shifted.X.mean(axis=0)).flatten()
    index_name_train = ['mean_expression_00hr_cyno_train','mean_expression_00hr_human_train','mean_expression_00hr_cyno_shifted_train']
    df_ct_mean_expr = pd.DataFrame([ mean_expression_00hr_cyno_train,
                                    mean_expression_00hr_human_train,
                                    mean_expression_00hr_cyno_shifted_train],index = index_name_train)

    df_ct_mean_expr.to_csv(os.path.join(base_table_path,pre + '_df_ct_mean_expr_train_' + str(celltype) + '.csv'))
    plt.figure(figsize=(6, 4.5))
    plt.scatter(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_human_train,c='#9eb4e0',edgecolors='k',alpha=0.7,s=50)
    plt.xlabel("Cyno Humanized", fontsize=12)
    plt.ylabel("Human", fontsize=12)
    plt.title('Mean correlation plot - ' + str(celltype), fontsize=14)
    plt.savefig(os.path.join(base_plots_path,pre +'_' +set_type +'_scatter_correlation_human_cyno_' + str(celltype) + '.pdf'),bbox_inches='tight')
    #plt.show()
    plt.close()

    plt.figure(figsize=(6, 4.5))
    plt.scatter(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_cyno_train,c='#9eb4e0',edgecolors='k',alpha=0.7,s=50)
    plt.xlabel("Cyno Humanized", fontsize=12)
    plt.ylabel("Cyno", fontsize=12)
    plt.title('Mean correlation plot - ' + str(celltype), fontsize=14)
    plt.savefig(os.path.join(base_plots_path,pre +'_' +set_type +'_scatter_correlation_cyno_cyno_' + str(celltype) + '.pdf'),bbox_inches='tight')
    #plt.show()
    plt.close()

    #R2 score
    r2_cyno_shifted_human_train = r2_score(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_human_train)
    r2_cyno_shifted_cyno_train = r2_score(mean_expression_00hr_cyno_shifted_train,mean_expression_00hr_cyno_train)
    r2_cyno_human_train = r2_score(mean_expression_00hr_cyno_train,mean_expression_00hr_human_train)

    columns_df = ['r2_cyno_shifted_human','r2_cyno_shifted_cyno','r2_cyno_human','celltype','set_type']
    df_r2_humanization_train = pd.DataFrame(columns = columns_df)
    df_r2_humanization_train  = df_r2_humanization_train.append(pd.DataFrame([r2_cyno_shifted_human_train,r2_cyno_shifted_cyno_train,r2_cyno_human_train,celltype,set_type],index=columns_df).T) 

    set_type = 'test'
    adata_test_00hr = hscg.filter_multiple_adata_obs(adata=adata_test_all_ct,
                                                         col_names=['cluster_azimut1_5_scanvi_v2','timepoint'],
                                                         val_names=[celltype,'00hr'])
    adata_test_00hr_cyno = hscg.filter_multiple_adata_obs(adata=adata_test_all_ct,
                                                         col_names=['cluster_azimut1_5_scanvi_v2','timepoint','species'],
                                                         val_names=[celltype,'00hr','cyno'])
    adata_test_00hr_human = hscg.filter_multiple_adata_obs(adata=adata_test_all_ct,
                                                         col_names=['cluster_azimut1_5_scanvi_v2','timepoint','species'],
                                                         val_names=[celltype,'00hr','human'])

    latent_adata_test_00hr = hscg.get_latent_representation_object(model=model,adata=adata_test_00hr)
    latent_adata_test_00hr_human = hscg.filter_multiple_adata_obs(adata=latent_adata_test_00hr ,
                                                                       col_names=['timepoint','species'],
                                                                       val_names=['00hr','human'])
    latent_adata_test_00hr_cyno = hscg.filter_multiple_adata_obs(adata=latent_adata_test_00hr ,
                                                                   col_names=['timepoint','species'],
                                                                   val_names=['00hr','cyno'])

    latent_adata_test_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_test_00hr_cyno,
                                                                           delta=delta_cross_species_train,
                                                                           cols_suffix=['timepoint','species'],
                                                                           suffix=['_shifted','_shifted'])

    adata_latent_comp_test = ad.concat([latent_adata_test_00hr_human,latent_adata_test_00hr_cyno,latent_adata_test_00hr_cyno_shifted])
    h.prepare_umap(adata_latent_comp_test)
    h.umap_plot_v2(adata=adata_latent_comp_test,
                 color='species',
                 base_plots_path = base_plots_path,
                 plot_name = pre +'_'+ set_type +'_umap_latent' + str(celltype) + '.pdf',
                palette = palette_species_shifted) 

    latent_adata_test_00hr_cyno = hscg.get_latent_representation_object(model=model,adata= adata_test_00hr_cyno)

    latent_adata_test_00hr_cyno_shifted = hscg.shift_adata_in_latent_space_v2(adata_base=latent_adata_test_00hr_cyno,delta=delta_cross_species_train)

    decoded_adata_test_00hr_cyno_shifted =hscg.decode(model=model,
                                              latent_adata=latent_adata_test_00hr_cyno_shifted,
                                              input_adata=adata_test_00hr_cyno) 

    decoded_adata_test_00hr_cyno_shifted.obs['timepoint'] = decoded_adata_test_00hr_cyno_shifted.obs['timepoint'].astype(str) + '_shifted'
    decoded_adata_test_00hr_cyno_shifted.obs['species'] = decoded_adata_test_00hr_cyno_shifted.obs['species'].astype(str) + '_shifted'

    adata_test_00hr_decoded = ad.concat([adata_test_00hr_cyno,adata_test_00hr_human,decoded_adata_test_00hr_cyno_shifted])

    h.prepare_umap(adata_test_00hr_decoded)

    palette = ['#178f92','#175f5d',"#d8c29d"]
    h.umap_plot_v2(adata=adata_test_00hr_decoded,
                 color='species',
                 base_plots_path = base_plots_path,
                 plot_name = pre +'_'+ set_type +'_umap_GE_' + str(celltype) + '.pdf',
                palette = palette_species_shifted) 

    mean_expression_00hr_cyno_test = np.array(adata_test_00hr_cyno.X.mean(axis=0)).flatten()
    mean_expression_00hr_human_test = np.array(adata_test_00hr_human.X.mean(axis=0)).flatten()
    mean_expression_00hr_cyno_shifted_test = np.array(decoded_adata_test_00hr_cyno_shifted.X.mean(axis=0)).flatten()
    index_name_test = ['mean_expression_00hr_cyno_test','mean_expression_00hr_human_test','mean_expression_00hr_cyno_shifted_test']
    df_ct_mean_expr = pd.DataFrame([ mean_expression_00hr_cyno_test,
                                    mean_expression_00hr_human_test,
                                    mean_expression_00hr_cyno_shifted_test],index = index_name_test)
    df_ct_mean_expr.to_csv(os.path.join(base_table_path,pre + '_df_ct_mean_expr_test_' + str(celltype) + '.csv'))
    plt.figure(figsize=(6, 4.5))
    plt.scatter(mean_expression_00hr_cyno_shifted_test,mean_expression_00hr_human_test,c='#9eb4e0',edgecolors='k',alpha=0.7,s=50)
    plt.xlabel("Cyno Humanized", fontsize=12)
    plt.ylabel("Human", fontsize=12)
    plt.title('Mean correlation plot - ' + str(celltype), fontsize=14)
    plt.savefig(os.path.join(base_plots_path,pre +'_' +set_type + '_scatter_correlation_human_cyno_' + str(celltype) + '.pdf'),bbox_inches='tight')
    #plt.show()
    plt.close()

    plt.figure(figsize=(6, 4.5))
    plt.scatter(mean_expression_00hr_cyno_shifted_test,mean_expression_00hr_cyno_test,c='#9eb4e0',edgecolors='k',alpha=0.7,s=50)
    plt.xlabel("Cyno Humanized", fontsize=12)
    plt.ylabel("Cyno", fontsize=12)
    plt.title('Mean correlation plot - ' + str(celltype), fontsize=14)
    plt.savefig(os.path.join(base_plots_path,pre +'_' +set_type + '_scatter_correlation_cyno_cyno_' + str(celltype) + '.pdf'),bbox_inches='tight')
    #plt.show()
    plt.close()

    #R2 score
    r2_cyno_shifted_human_test = r2_score(mean_expression_00hr_cyno_shifted_test,mean_expression_00hr_human_test)
    r2_cyno_shifted_cyno_test = r2_score(mean_expression_00hr_cyno_shifted_test,mean_expression_00hr_cyno_test)
    r2_cyno_human_test = r2_score(mean_expression_00hr_cyno_test,mean_expression_00hr_human_test)

    columns_df = ['r2_cyno_shifted_human','r2_cyno_shifted_cyno','r2_cyno_human','celltype','set_type']
    df_r2_humanization_test = pd.DataFrame(columns = columns_df)
    df_r2_humanization_test  = df_r2_humanization_test.append(pd.DataFrame([r2_cyno_shifted_human_test,r2_cyno_shifted_cyno_test,r2_cyno_human_test,celltype,set_type],index=columns_df).T) 

    # save all
    df_r2_humanization_train_test = df_r2_humanization_test.append(df_r2_humanization_train)
    save_r2_df_train_test = os.path.join(base_table_path,pre + '_r2_scores_train_test' + '_'+ str(celltype) + '.csv')
    df_r2_humanization_train_test.to_csv(save_r2_df_train_test)

    #save anndata objects

    adata_train_00hr_decoded.write_h5ad(
        os.path.join(base_anndata_objects,pre + '_' + str(celltype) + '_adata_train_00hr_decoded.h5ad')
    )

    adata_test_00hr_decoded.write_h5ad(
        os.path.join(base_anndata_objects,pre + '_' + str(celltype) + '_adata_test_00hr_decoded.h5ad')
    )

    adata_latent_comp_train.write_h5ad(
        os.path.join(base_anndata_objects,pre + '_' + str(celltype) + '_adata_latent_comp_00hr_train.h5ad')
    )

    adata_latent_comp_test.write_h5ad(
        os.path.join(base_anndata_objects,pre + '_' + str(celltype) + '_adata_latent_comp_00hr_test.h5ad')
    )

In [ ]:

base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)